# Sentence Retrieval

In [3]:
import json,math,nltk

train_file = open("train.json",'r')
dev_file = open("dev.json",'r')
test_file=open("dev.json",'r')
train = json.loads(train_file.read())
dev = json.loads(dev_file.read())
test = json.loads(test_file.read())
# train[0]['sentences']
# train[0]['qa']

In [4]:
from nltk.corpus import stopwords
# may be stem?
# stop word
# tf-idf inverse_transform

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
stopword =  stopwords.words()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def doc_word_dict(doc):
    word_dict = set()
    for sent in doc['sentences']:
        for word in  word_tokenizer.tokenize(sent):
            word = lemmatize(word.lower())
            if word not in stopword:
                word_dict.add(word)
    return word_dict

In [5]:
def get_BOW(sent):
    term_dict={}
    for word in word_tokenizer.tokenize(sent):
        word = lemmatize(word.lower())
        if word not in stopword:
            term_dict[word]=term_dict.setdefault(word,0)+1
    return term_dict

def cal_BOW(doc):
    doc_term_matrix = [] 
    for sent in doc['sentences']:
        temp = get_BOW(sent)
        doc_term_matrix.append(temp)
    return doc_term_matrix

In [4]:
# def BOW_tf(sent,word_dict):
# #     term_dict = dict.fromkeys(word_dict,0)
#     term_dict={}
#     for word in word_tokenizer.tokenize(sent):
#         word = lemmatize(word.lower())
#         if word in word_dict:
#             term_dict[word]=term_dict.setdefault(word,0)+1
#     `term_dict = {k: (v +0.0)/ max(term_dict.values()) for k, v in term_dict.items()}
#     for word in term_dict:
#         word_dict[word] = word_dict[word]+1
#     return term_dict,word_dict

# def cal_BOW(doc):
#     word_dict = dict.fromkeys(doc_word_dict(train[0]),0)
#     doc_term_matrix = []
#     for sent in doc['sentences']:
#         temp, word_dict = BOW_tf(sent,word_dict)
#         doc_term_matrix.append(temp)
#     return doc_term_matrix,word_dict

# tf,idf = cal_BOW(train[0])
# idf = {k:math.log(len(train[0]['sentences'])/(v+1.0)) for k,v in idf.items()}

# tfidf = tf[:]
# for i in range(len(tf)):
#     for k,v in tf[i].items():
#         tf[i][k] = v*idf[k]

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance

vectorizer = DictVectorizer()
transformer = TfidfTransformer(smooth_idf=False,norm=None)

term_matrix = transformer.fit_transform(vectorizer.fit_transform(cal_BOW(train[0])))
query =  transformer.transform(vectorizer.transform(get_BOW(train[0]['qa'][0]['question'])))

In [7]:
def get_best_doc_num(query):
    query =  transformer.transform(vectorizer.transform(get_BOW(query)))
    f = lambda x: cos_distance(query.toarray(),term_matrix[x].toarray())
    best_doc = min(range(term_matrix.shape[0]),key=f)
    return best_doc

In [8]:
correct_rate = []
for doc in test:
    doc_correct_rate = []
    term_matrix = transformer.fit_transform(vectorizer.fit_transform(cal_BOW(doc)))
    for qa in doc['qa']:
        doc_correct_rate.append(get_best_doc_num(qa['question']))
    correct_rate.append(doc_correct_rate)

KeyboardInterrupt: 

In [85]:
set(correct_rate[0])

{0,
 1,
 2,
 3,
 5,
 6,
 8,
 10,
 12,
 13,
 14,
 17,
 18,
 19,
 22,
 23,
 24,
 26,
 28,
 30,
 31,
 35,
 36,
 37,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 52,
 53,
 55,
 56,
 58,
 59,
 61,
 63,
 64,
 65,
 67,
 68,
 71,
 72,
 75,
 76,
 77,
 78,
 81,
 85,
 86,
 87,
 88,
 89,
 91,
 93}

# Entity Extraction

In [18]:
from nltk.tag.stanford import StanfordNERTagger
st = StanfordNERTagger('/usr/share/stanford-ner/classifiers/english.muc.7class.distsim.crf.ser.gz',
               '/usr/share/stanford-ner/stanford-ner.jar') 
st.tag(word_tokenizer.tokenize("Located in the northwestern Pacific Ocean, Guam is one of five American territories with an established civilian government in 1980s."))
# st.tag("Located in the northwestern Pacific Ocean, Guam is one of five American territories with an established civilian government in 1980s.".split())

[(u'Located', u'O'),
 (u'in', u'O'),
 (u'the', u'O'),
 (u'northwestern', u'O'),
 (u'Pacific', u'LOCATION'),
 (u'Ocean', u'LOCATION'),
 (u',', u'O'),
 (u'Guam', u'LOCATION'),
 (u'is', u'O'),
 (u'one', u'O'),
 (u'of', u'O'),
 (u'five', u'O'),
 (u'American', u'O'),
 (u'territories', u'O'),
 (u'with', u'O'),
 (u'an', u'O'),
 (u'established', u'O'),
 (u'civilian', u'O'),
 (u'government', u'O'),
 (u'in', u'O'),
 (u'1980s', u'DATE'),
 (u'.', u'O')]

In [88]:
import time

start = time.time()
test_sent_list = []
for i in range(len(correct_rate)):
    for j in list(set(correct_rate[i])):
        tmp = st.tag(test[i]['sentences'][j].split())
        test_sent_list.append(tmp)
print time.time()-start

452.405480146


In [89]:
start = time.time()
test_sent_list = []
for i in range(len(correct_rate)):
    for j in list(set(correct_rate[i])):
        test_sent_list.append(test[i]['sentences'][j].split())
test_sent_list = st.tag_sents(test_sent_list) 
print time.time()-start

3.00301289558


In [90]:
test_sent_list

[[(u'Infrared', u'O'),
  (u'radiation', u'O'),
  (u'is', u'O'),
  (u'used', u'O'),
  (u'in', u'O'),
  (u'industrial,', u'O'),
  (u'scientific,', u'O'),
  (u'and', u'O'),
  (u'medical', u'O'),
  (u'applications.', u'O')],
 [(u'Night-vision', u'O'),
  (u'devices', u'O'),
  (u'using', u'O'),
  (u'active', u'O'),
  (u'near-infrared', u'O'),
  (u'illumination', u'O'),
  (u'allow', u'O'),
  (u'people', u'O'),
  (u'or', u'O'),
  (u'animals', u'O'),
  (u'to', u'O'),
  (u'be', u'O'),
  (u'observed', u'O'),
  (u'without', u'O'),
  (u'the', u'O'),
  (u'observer', u'O'),
  (u'being', u'O'),
  (u'detected.', u'O')],
 [(u'Infrared', u'O'),
  (u'astronomy', u'O'),
  (u'uses', u'O'),
  (u'sensor-equipped', u'O'),
  (u'telescopes', u'O'),
  (u'to', u'O'),
  (u'penetrate', u'O'),
  (u'dusty', u'O'),
  (u'regions', u'O'),
  (u'of', u'O'),
  (u'space,', u'O'),
  (u'such', u'O'),
  (u'as', u'O'),
  (u'molecular', u'O'),
  (u'clouds;', u'O'),
  (u'detect', u'O'),
  (u'objects', u'O'),
  (u'such', u'O'),
  (

In [21]:
for i in range(len(correct_rate)):
    for j in range(len(correct_rate[i])):
        st.tag((test[i]['sentences'][correct_rate[i][j]]).split()):
        if k2 == 'PERSON':
            print k1,k2
        elif k2 == 'LOCATION':
            print k1,k2
        elif k2 == 'DATE' or k2 == 'TIME' or k2 == 'PERCENT':
            print k1,k2
        elif j!=0 and k2 == 'O' and k1[0].isupper():
            print k1,k2

[[(u'I', u'O'), (u'n', u'O'), (u'f', u'O'), (u'r', u'O'), (u'a', u'O'), (u'r', u'O'), (u'e', u'O'), (u'd', u'O'), (u'r', u'O'), (u'a', u'O'), (u'd', u'O'), (u'i', u'O'), (u'a', u'O'), (u't', u'O'), (u'i', u'O'), (u'o', u'O'), (u'n', u'O'), (u'i', u'O'), (u's', u'O'), (u'u', u'O'), (u's', u'O'), (u'e', u'O'), (u'd', u'O'), (u'i', u'O'), (u'n', u'O'), (u'i', u'O'), (u'n', u'O'), (u'd', u'O'), (u'u', u'O'), (u's', u'O'), (u't', u'O'), (u'r', u'O'), (u'i', u'O'), (u'a', u'O'), (u'l', u'O'), (u',', u'O'), (u's', u'O'), (u'c', u'O'), (u'i', u'O'), (u'e', u'O'), (u'n', u'O'), (u't', u'O'), (u'i', u'O'), (u'f', u'O'), (u'i', u'O'), (u'c', u'O'), (u',', u'O'), (u'a', u'O'), (u'n', u'O'), (u'd', u'O'), (u'm', u'O'), (u'e', u'O'), (u'd', u'O'), (u'i', u'O'), (u'c', u'O'), (u'a', u'O'), (u'l', u'O'), (u'a', u'O'), (u'p', u'O'), (u'p', u'O'), (u'l', u'O'), (u'i', u'O'), (u'c', u'O'), (u'a', u'O'), (u't', u'O'), (u'i', u'O'), (u'o', u'O'), (u'n', u'O'), (u's', u'O'), (u'.', u'O'), (u'N', u'O'), (u'i

[[(u'I', u'O'), (u'n', u'O'), (u'f', u'O'), (u'r', u'O'), (u'a', u'O'), (u'r', u'O'), (u'e', u'O'), (u'd', u'O'), (u'r', u'O'), (u'a', u'O'), (u'd', u'O'), (u'i', u'O'), (u'a', u'O'), (u't', u'O'), (u'i', u'O'), (u'o', u'O'), (u'n', u'O'), (u'i', u'O'), (u's', u'O'), (u'u', u'O'), (u's', u'O'), (u'e', u'O'), (u'd', u'O'), (u'i', u'O'), (u'n', u'O'), (u'i', u'O'), (u'n', u'O'), (u'd', u'O'), (u'u', u'O'), (u's', u'O'), (u't', u'O'), (u'r', u'O'), (u'i', u'O'), (u'a', u'O'), (u'l', u'O'), (u',', u'O'), (u's', u'O'), (u'c', u'O'), (u'i', u'O'), (u'e', u'O'), (u'n', u'O'), (u't', u'O'), (u'i', u'O'), (u'f', u'O'), (u'i', u'O'), (u'c', u'O'), (u',', u'O'), (u'a', u'O'), (u'n', u'O'), (u'd', u'O'), (u'm', u'O'), (u'e', u'O'), (u'd', u'O'), (u'i', u'O'), (u'c', u'O'), (u'a', u'O'), (u'l', u'O'), (u'a', u'O'), (u'p', u'O'), (u'p', u'O'), (u'l', u'O'), (u'i', u'O'), (u'c', u'O'), (u'a', u'O'), (u't', u'O'), (u'i', u'O'), (u'o', u'O'), (u'n', u'O'), (u's', u'O'), (u'.', u'O'), (u'N', u'O'), (u'i

[[(u'I', u'O'), (u'n', u'O'), (u'f', u'O'), (u'r', u'O'), (u'a', u'O'), (u'r', u'O'), (u'e', u'O'), (u'd', u'O'), (u'r', u'O'), (u'a', u'O'), (u'd', u'O'), (u'i', u'O'), (u'a', u'O'), (u't', u'O'), (u'i', u'O'), (u'o', u'O'), (u'n', u'O'), (u'i', u'O'), (u's', u'O'), (u'u', u'O'), (u's', u'O'), (u'e', u'O'), (u'd', u'O'), (u'i', u'O'), (u'n', u'O'), (u'i', u'O'), (u'n', u'O'), (u'd', u'O'), (u'u', u'O'), (u's', u'O'), (u't', u'O'), (u'r', u'O'), (u'i', u'O'), (u'a', u'O'), (u'l', u'O'), (u',', u'O'), (u's', u'O'), (u'c', u'O'), (u'i', u'O'), (u'e', u'O'), (u'n', u'O'), (u't', u'O'), (u'i', u'O'), (u'f', u'O'), (u'i', u'O'), (u'c', u'O'), (u',', u'O'), (u'a', u'O'), (u'n', u'O'), (u'd', u'O'), (u'm', u'O'), (u'e', u'O'), (u'd', u'O'), (u'i', u'O'), (u'c', u'O'), (u'a', u'O'), (u'l', u'O'), (u'a', u'O'), (u'p', u'O'), (u'p', u'O'), (u'l', u'O'), (u'i', u'O'), (u'c', u'O'), (u'a', u'O'), (u't', u'O'), (u'i', u'O'), (u'o', u'O'), (u'n', u'O'), (u's', u'O'), (u'.', u'O'), (u'N', u'O'), (u'i

[[(u'I', u'O'), (u'n', u'O'), (u'f', u'O'), (u'r', u'O'), (u'a', u'O'), (u'r', u'O'), (u'e', u'O'), (u'd', u'O'), (u'r', u'O'), (u'a', u'O'), (u'd', u'O'), (u'i', u'O'), (u'a', u'O'), (u't', u'O'), (u'i', u'O'), (u'o', u'O'), (u'n', u'O'), (u'i', u'O'), (u's', u'O'), (u'u', u'O'), (u's', u'O'), (u'e', u'O'), (u'd', u'O'), (u'i', u'O'), (u'n', u'O'), (u'i', u'O'), (u'n', u'O'), (u'd', u'O'), (u'u', u'O'), (u's', u'O'), (u't', u'O'), (u'r', u'O'), (u'i', u'O'), (u'a', u'O'), (u'l', u'O'), (u',', u'O'), (u's', u'O'), (u'c', u'O'), (u'i', u'O'), (u'e', u'O'), (u'n', u'O'), (u't', u'O'), (u'i', u'O'), (u'f', u'O'), (u'i', u'O'), (u'c', u'O'), (u',', u'O'), (u'a', u'O'), (u'n', u'O'), (u'd', u'O'), (u'm', u'O'), (u'e', u'O'), (u'd', u'O'), (u'i', u'O'), (u'c', u'O'), (u'a', u'O'), (u'l', u'O'), (u'a', u'O'), (u'p', u'O'), (u'p', u'O'), (u'l', u'O'), (u'i', u'O'), (u'c', u'O'), (u'a', u'O'), (u't', u'O'), (u'i', u'O'), (u'o', u'O'), (u'n', u'O'), (u's', u'O'), (u'.', u'O'), (u'N', u'O'), (u'i

KeyboardInterrupt: 

# Answer Ranking

In [60]:
import gensim
from gensim import corpora
import math

class BM25 :
    def __init__(self, fn_docs, delimiter='|') :
        self.dictionary = corpora.Dictionary()
        self.DF = {}
        self.delimiter = delimiter
        self.DocTF = []
        self.DocIDF = {}
        self.N = 0
        self.DocAvgLen = 0
        self.fn_docs = fn_docs
        self.DocLen = []
        self.buildDictionary()
        self.TFIDF_Generator()

    def buildDictionary(self) :
        raw_data = []
        for line in file(self.fn_docs) :
            raw_data.append(line.strip().split(self.delimiter))
        self.dictionary.add_documents(raw_data)

    def TFIDF_Generator(self, base=math.e) :
        docTotalLen = 0
        for line in file(self.fn_docs) :
            doc = line.strip().split(self.delimiter)
            docTotalLen += len(doc)
            self.DocLen.append(len(doc))
            #print self.dictionary.doc2bow(doc)
            bow = dict([(term, freq*1.0/len(doc)) for term, freq in self.dictionary.doc2bow(doc)])
            for term, tf in bow.items() :
                if term not in self.DF :
                    self.DF[term] = 0
                self.DF[term] += 1
            self.DocTF.append(bow)
            self.N = self.N + 1
        for term in self.DF:
            self.DocIDF[term] = math.log((self.N - self.DF[term] +0.5) / (self.DF[term] + 0.5), base)
        self.DocAvgLen = docTotalLen / self.N

    def BM25Score(self, Query=[], k1=1.5, b=0.75) :
        query_bow = self.dictionary.doc2bow(Query)
        scores = []
        for idx, doc in enumerate(self.DocTF) :
            commonTerms = set(dict(query_bow).keys()) & set(doc.keys())
            tmp_score = []
            doc_terms_len = self.DocLen[idx]
            for term in commonTerms :
                upper = (doc[term] * (k1+1))
                below = ((doc[term]) + k1*(1 - b + b*doc_terms_len/self.DocAvgLen))
                tmp_score.append(self.DocIDF[term] * upper / below)
            scores.append(sum(tmp_score))
        return scores

    def TFIDF(self) :
        tfidf = []
        for doc in self.DocTF :
            doc_tfidf  = [(term, tf*self.DocIDF[term]) for term, tf in doc.items()]
            doc_tfidf.sort()
            tfidf.append(doc_tfidf)
        return tfidf

    def Items(self) :
        # Return a list [(term_idx, term_desc),]
        items = self.dictionary.items()
        items.sort()
        return items

if __name__ == '__main__' :
    fn_docs = 'mycorpus.txt'
    bm25 = BM25(fn_docs, delimiter=' ')
    Query = 'The intersection graph of paths in trees survey Graph'
    Query = Query.split()
    scores = bm25.BM25Score(Query)
    print scores
    tfidf = bm25.TFIDF()
#     print bm25.Items()
    for i, tfidfscore in enumerate(tfidf):
        print i, tfidfscore

[0, -0.00993319335279988, 0.1712756703100223, -0.10833186147108911, -0.08897894166003212, 0.13457374095787467, 1.642922484773619, 0.15791141726235663, 1.0831388761516576]
0 [(0, 0.2168251319235133), (1, 0.2168251319235133), (2, 0.2168251319235133), (3, 0.2168251319235133), (4, 0.2168251319235133), (5, 0.13732653608351372), (6, 0.2168251319235133), (7, 0.13732653608351372)]
1 [(5, 0.10986122886681099), (8, 0.10986122886681099), (9, -0.1238078416812447), (10, 0.06190392084062235), (11, 0.10986122886681099), (12, 0.06190392084062235), (13, 0.10986122886681099), (14, 0.17346010553881064), (15, 0.10986122886681099)]
2 [(7, 0.1831020481113516), (10, 0.10317320140103725), (12, 0.10317320140103725), (16, 0.10317320140103725), (17, 0.28910017589801773), (18, 0.1831020481113516)]
3 [(9, -0.07737990105077794), (10, 0.07737990105077794), (18, 0.13732653608351372), (19, 0.13732653608351372), (20, 0.2168251319235133), (21, 0.2168251319235133), (22, 0.2168251319235133), (23, 0.2168251319235133)]
4 [(